In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('cleaned_data.csv')

## Adding 100 instances from test data so that our model can learn from it

In [ ]:
cc=pd.read_csv("/content/CCAC_train_data.csv")
#instances from the test data

In [ ]:
cc=cc[['Subject','Body','Label','SenderEmailAddress']]

In [ ]:
train_data=train_data[['Subject','body','Label','From']]
train_data

,Subject,body,Label,From
0,continental.com specials for chris germany,"b""continental.com specials for chris germany\n...",0,continental_airlines_inc@coair.rsc01.com
1,re: rebuttal testimony (draft) on behalf of tw,"b""fyi - i have not changes to suggest.\n\nmich...",0,alexanms@sce.com
2,Scheduled Maintenance & Upgrade,"\nit service desk\n\nattn account user,\n\nsch...",1,tech.team@tech-center.com
3,re: texas legislative oversight committee,"b'i have spoken to mark, he will send a list o...",0,jean.ryall@enron.com
4,correspondence from donn fullenweider,b'\nsee attached correspondence from donn ful...,0,jls@fullenweider.com
...,...,...,...,...
10891,Receipt for Your Payment,"\r\n\r\ndear paypal member ,\r\...",1,security@PayPal.com
10892,re: e-mail alert and testimony finalization,"b""i am resending this because many of you have...",0,bcherry@sempra.com
10893,re: offer on gas daily option at waha and perm,b' \n\n -----original message-----\nfrom: \tbu...,0,larry.may@enron.com
10894,re:,"b""can we make it a weekend\n\n -----original m...",0,mike.maggi@enron.com


In [ ]:
train_data = train_data.rename(columns={'body': 'Body','From':'SenderEmailAddress'})


# Concatenating ccac and train data




In [ ]:
frames = [train_data, cc]
train = pd.concat(frames)

In [ ]:
train.sample(frac=1)
#shuffling the data

,Subject,Body,Label,SenderEmailAddress
8546,re: esa facility agreement,"b'brett,\n\ni think friday is perfect. i will...",0,kay.mann@enron.com
10053,fpl - enron isda agreements & attached files,b'sara and jason: the changes you last sent w...,0,sgm@steelhector.com
3978,enron - livingston county privileged and confi...,"b'matt, since so much of this related to tax, ...",0,kay.mann@enron.com
8695,AOL Administration Center Notification #72559,[http://csi.web.aol.com/email/aolmail/header.g...,1,AOLCenter@message.aol.com
2376,Your mailbox has exceeded the storage limit,your mailbox has exceeded the storage limit wh...,1,ssanborn@fpschools.org
...,...,...,...,...
10856,0002 variances,"b'here is my breakdown for the number of 890,0...",0,phillip.love@enron.com
10419,fw: good luck america,b'----- forwarded by tana jones/hou/ect on 11/...,0,tana.jones@enron.com
9453,inventory,"b"" under section four it shows where we cashed...",0,germanj@basf-corp.com
6832,re: deal 524310,"b'surprise, surprise - mike checked with his b...",0,kate.symes@enron.com


In [ ]:
train.reset_index(inplace=True)

### Extracting URLs from the body

In [ ]:
import re
def extract_url(text):
    urls = re.findall('(?:http\:|https\:)?//(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text)
    return urls

In [ ]:
url_list = []
for i in range(len(train)):
    text=str(train['Body'][i])
    urls=extract_url(text)
    url_list.append(urls)
train['url']=url_list

## Preprocessing



### Removing emails, links and other escape sequences from the Body after the new data was added

In [ ]:
def text_clean(email):
   
    email = re.sub('[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+',' ',email) #email IDs
    email = re.sub(r"http\S+", " ", email) #links
    email = re.sub(r"www\S+", " ", email) #links
    email = re.sub('\n', ' ', email)
    email = re.sub('\r', ' ', email)
    email = re.sub('\\t', ' ', email)
    email = email.replace("'b"," ")
    email = ''.join([i for i in email if not i.isdigit()])
    #ignoring the digits and joining the rest
    words = email.split()
    table = str.maketrans(' ', ' ', '.!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~—‘’”“')# removing special characters
    #maps character to replacement
    stripped = [w.translate(table) for w in words]
    email=" ".join(stripped)
    return email



In [ ]:
tcl=[]
for i in range(len(train)):
    tc=str(train['Body'][i])
    tc=remove_esc(tc)
    tcl.append(tc)
train['clean_esc_body']=tcl


In [ ]:
train

,index,Subject,Body,Label,SenderEmailAddress,url,clean_esc_body
0,0,continental.com specials for chris germany,"b""continental.com specials for chris germany\n...",0,continental_airlines_inc@coair.rsc01.com,"[http://continentalairlines.rsc01.net, http://...",continental.com specials for chris germany wed...
1,1,re: rebuttal testimony (draft) on behalf of tw,"b""fyi - i have not changes to suggest.\n\nmich...",0,alexanms@sce.com,[],fyi i have not changes to suggest. michael ...
2,2,Scheduled Maintenance & Upgrade,"it service desk attn account user, schedule...",1,tech.team@tech-center.com,[],"it service desk attn account user, schedule..."
3,3,re: texas legislative oversight committee,"b'i have spoken to mark, he will send a list o...",0,jean.ryall@enron.com,[],"i have spoken to mark, he will send a list of ..."
4,4,correspondence from donn fullenweider,b'\nsee attached correspondence from donn ful...,0,jls@fullenweider.com,[],see attached correspondence from donn fullen...
...,...,...,...,...,...,...,...
10992,96,FW: [EXT]Invitation: Midcontinent Independent ...,-----Original Appointment----- From...,0,AkcaliBra3136@ccac.sales.com,"[https://walk.zoom.us, https://walk.zoom.us, h...",Original Appointment From...
10993,97,RE: Feedback on my list,From: Osamah Achard < > Sent: Frid...,0,AkcaliBra3136@ccac.sales.com,"[http://www.ccac, http://twitter.com, http://w...",From: Osamah Achard < > Sent: Frid...
10994,98,ccac sales,"Hello, There's an ongoing survey recruitment ...",1,AhluwaliaNor4933@ccac.sales.com,[https://docs.google.com],"Hello, There s an ongoing survey recruitment ..."
10995,99,Accepted: BMIS,nan,0,AkcaliBra3136@ccac.sales.com,[],nan


cleaned the escaped characters and removed the starting b

### Extracting Domain Names

In [ ]:
train.isna().sum()

index                   0
Subject               284
Body                    0
Label                   0
SenderEmailAddress     15
url                     0
clean_esc_body          0
dtype: int64

Dropping where there is no sender name

In [ ]:
null_from_index = train.loc[train['SenderEmailAddress'].isnull() == True].index
train.drop(null_from_index, axis = 0, inplace = True)

### Domain List

In [ ]:
def extract_domain(string):
    domain = str(string).split('@')[1].split('.')[0]
    return domain
# returns the sub-domain. split by @ and get the part after the '@'.   
# next split returns the part before the '.' which gives us the subdomain.
# eg meboin@iu.edu will give us first iu.edu after splitting at @. then iu after splitting at .

In [ ]:
domain_list = []
for email in train['SenderEmailAddress']:
    try:
        email_domain = extract_domain(email)
        domain_list.append(email_domain)
    except:
        print("Row that throws error is:")
        print(train[email])

In [ ]:
train['Domain_from']=domain_list

In [ ]:
train

,index,Subject,Body,Label,SenderEmailAddress,url,clean_esc_body,Domain_from
0,0,continental.com specials for chris germany,"b""continental.com specials for chris germany\n...",0,continental_airlines_inc@coair.rsc01.com,"[http://continentalairlines.rsc01.net, http://...",continental.com specials for chris germany wed...,coair
1,1,re: rebuttal testimony (draft) on behalf of tw,"b""fyi - i have not changes to suggest.\n\nmich...",0,alexanms@sce.com,[],fyi i have not changes to suggest. michael ...,sce
2,2,Scheduled Maintenance & Upgrade,"it service desk attn account user, schedule...",1,tech.team@tech-center.com,[],"it service desk attn account user, schedule...",tech-center
3,3,re: texas legislative oversight committee,"b'i have spoken to mark, he will send a list o...",0,jean.ryall@enron.com,[],"i have spoken to mark, he will send a list of ...",enron
4,4,correspondence from donn fullenweider,b'\nsee attached correspondence from donn ful...,0,jls@fullenweider.com,[],see attached correspondence from donn fullen...,fullenweider
...,...,...,...,...,...,...,...,...
10992,96,FW: [EXT]Invitation: Midcontinent Independent ...,-----Original Appointment----- From...,0,AkcaliBra3136@ccac.sales.com,"[https://walk.zoom.us, https://walk.zoom.us, h...",Original Appointment From...,ccac
10993,97,RE: Feedback on my list,From: Osamah Achard < > Sent: Frid...,0,AkcaliBra3136@ccac.sales.com,"[http://www.ccac, http://twitter.com, http://w...",From: Osamah Achard < > Sent: Frid...,ccac
10994,98,ccac sales,"Hello, There's an ongoing survey recruitment ...",1,AhluwaliaNor4933@ccac.sales.com,[https://docs.google.com],"Hello, There s an ongoing survey recruitment ...",ccac
10995,99,Accepted: BMIS,nan,0,AkcaliBra3136@ccac.sales.com,[],nan,ccac


### Extracting the Subject Labels



In [ ]:
import re
def remove_prefix(sub):
  pref_encoding=[]
  sub=sub.strip().lower()
  l=len(sub)
  x1=re.search('^re:',sub)
  #matching at the start of the string
  if x1!=None:
    label=1
    sub=sub[3:].strip()
    #splitting the string after the're'
    while True:# while loop to strip other labels if found since re can be suffixed with other labels
      if (re.search('^re:',sub)!=None):
        sub=sub[3:].strip()
        continue
      elif (re.search('^fw:',sub)!=None):
        sub=sub[3:].strip()
        continue
      elif (re.search('^[ext]',sub)!=None):
        sub=sub[5:].strip()
        continue
      elif (re.search('^[bulk]',sub)!=None):
        sub=sub[6:].strip()
        continue
      else:
        return sub,label
        break
  elif (re.search('^fw:',sub)!=None): # if first label found is forwarded then give label 2
    label=2
    sub=sub[3:].strip()
    while True: # proceeding with removing all other labels
      if (re.search('^re:',sub)!=None):
        sub=sub[3:].strip()
        continue
      elif (re.search('^fw:',sub)!=None):
        sub=sub[3:].strip()
        continue
      elif (re.search('^[ext]',sub)!=None):
        sub=sub[5:].strip()
        continue
      elif (re.search('^[bulk]',sub)!=None):
        sub=sub[6:].strip()
        continue
      else:
        return sub,label
        break
  elif (re.search('^\[ext]',sub)!=None):
    label=3
    sub=sub[5:].strip()
    while True:
      if (re.search('^re:',sub)!=None):
        sub=sub[3:].strip()
        continue
      elif (re.search('^fw:',sub)!=None):
        sub=sub[3:].strip()
        continue
      elif (re.search('^[ext]',sub)!=None):
        sub=sub[5:].strip()
        continue
      elif (re.search('^[bulk]',sub)!=None):
        sub=sub[6:].strip()
        continue
      else:
        return sub,label
        break
  else:
    label=0 # for all the cases give 0
    return sub,label
    

remove_prefix(" [ext]re: jkahjsndjkasd")

('jkahjsndjkasd', 3)

In [ ]:
clean_subject=[]
sub_pref_label=[]
for i in range(len(train)):
  
  x=str(train.iloc[i]['Subject'])
  a,b=remove_prefix(x)
  clean_subject.append(a)
  sub_pref_label.append(b)

In [ ]:
train['clean_subject']=clean_subject
train['sub_pref_label']=sub_pref_label

In [ ]:
train['sub_pref_label'].value_counts()

0    7980
1    2473
2     515
3      14
Name: sub_pref_label, dtype: int64

### one hot encoding subject prefix column

In [ ]:
train=pd.get_dummies(train,columns=['sub_pref_label'])
# to perform one hot encoding

In [ ]:
train

,Subject,Body,Label,SenderEmailAddress,url,clean_esc_body,Domain_from,clean_subject,sub_pref_label_0,sub_pref_label_1,sub_pref_label_2,sub_pref_label_3
0,continental.com specials for chris germany,"b""continental.com specials for chris germany\n...",0,continental_airlines_inc@coair.rsc01.com,"[http://continentalairlines.rsc01.net, http://...",continental.com specials for chris germany wed...,coair,continental.com specials for chris germany,1,0,0,0
1,re: rebuttal testimony (draft) on behalf of tw,"b""fyi - i have not changes to suggest.\n\nmich...",0,alexanms@sce.com,[],fyi i have not changes to suggest. michael ...,sce,rebuttal testimony (draft) on behalf of tw,0,1,0,0
2,Scheduled Maintenance & Upgrade,"it service desk attn account user, schedule...",1,tech.team@tech-center.com,[],"it service desk attn account user, schedule...",tech-center,scheduled maintenance & upgrade,1,0,0,0
3,re: texas legislative oversight committee,"b'i have spoken to mark, he will send a list o...",0,jean.ryall@enron.com,[],"i have spoken to mark, he will send a list of ...",enron,ative oversight committee,0,1,0,0
4,correspondence from donn fullenweider,b'\nsee attached correspondence from donn ful...,0,jls@fullenweider.com,[],see attached correspondence from donn fullen...,fullenweider,correspondence from donn fullenweider,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10992,FW: [EXT]Invitation: Midcontinent Independent ...,-----Original Appointment----- From...,0,AkcaliBra3136@ccac.sales.com,"[https://walk.zoom.us, https://walk.zoom.us, h...",Original Appointment From...,ccac,[ext]invitation: midcontinent independent syst...,0,0,1,0
10993,RE: Feedback on my list,From: Osamah Achard < > Sent: Frid...,0,AkcaliBra3136@ccac.sales.com,"[http://www.ccac, http://twitter.com, http://w...",From: Osamah Achard < > Sent: Frid...,ccac,feedback on my list,0,1,0,0
10994,ccac sales,"Hello, There's an ongoing survey recruitment ...",1,AhluwaliaNor4933@ccac.sales.com,[https://docs.google.com],"Hello, There s an ongoing survey recruitment ...",ccac,ccac sales,1,0,0,0
10995,Accepted: BMIS,nan,0,AkcaliBra3136@ccac.sales.com,[],nan,ccac,accepted: bmis,1,0,0,0


#Saving train preprocessed to csv

In [ ]:
train.to_csv("train_preprocessed.csv")

### Same Pipeline on testing data

In [ ]:
test=pd.read_csv("/content/CCAC_data.csv")

In [ ]:
test=test[['SenderEmailAddress','Subject','Body']]

In [ ]:
url_list = []
for i in range(len(test)):
    text=str(test['Body'][i])
    urls=extract_url(text)
    url_list.append(urls)
test['url']=url_list

In [ ]:
test['url']=url_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
cl=[]
for i in range(len(test)):
    tc=str(test['Body'][i])
    tc=remove_esc(tc)
    tcl.append(tc)
test['Body']=tcl




In [ ]:
esc=[]
for i in range(len(test)):
    st=str(test.iloc[i]['Body'])
    st=remove_esc(st)
    esc.append(st)
test['clean_esc_body']=esc

In [ ]:
domain_list = []
for i in range(len(test['SenderEmailAddress'])):
    try:
        email=str(test.iloc[i]['SenderEmailAddress'])
        email_domain = extract_domain(email)
        domain_list.append(email_domain)
    except:
        domain_list.append("NA")
        print("Row that throws error is:")
        print(i)

Row that throws error is:
405
Row that throws error is:
878
Row that throws error is:
1481
Row that throws error is:
1881
Row that throws error is:
2106
Row that throws error is:
2142
Row that throws error is:
2740
Row that throws error is:
3130
Row that throws error is:
3562
Row that throws error is:
3685
Row that throws error is:
4518
Row that throws error is:
4641
Row that throws error is:
4740


In [ ]:
test.iloc[405]['SenderEmailAddress']

'R STEPHENS'

In [ ]:
test['Domain_from']=domain_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
clean_subject=[]
sub_pref_label=[]
for i in range(len(test)):
  
    x=str(test.iloc[i]['Subject'])
    a,b=remove_prefix(x)
    clean_subject.append(a)
    sub_pref_label.append(b)

In [ ]:
test['clean_subject']=clean_subject
test['sub_pref_label']=sub_pref_label

In [ ]:
test=pd.get_dummies(test,columns=['sub_pref_label'])

In [ ]:
test.to_csv("test_preprocessed.csv")

# Model Building Starts Here

In [ ]:
trainf=train[['clean_subject','clean_esc_body']]

In [ ]:
testf=test[['clean_subject','clean_esc_body']]

### Loading the training and testing

In [ ]:
train=pd.read_csv("/content/train_preprocessed.csv")
test=pd.read_csv("/content/test_preprocessed.csv")



In [ ]:
train.isnull().sum()

Unnamed: 0              0
Subject               284
Body                    2
Label                   0
SenderEmailAddress      0
url                     0
clean_esc_body          2
Domain_from             0
clean_subject           0
sub_pref_label_0        0
sub_pref_label_1        0
sub_pref_label_2        0
sub_pref_label_3        0
dtype: int64

removing null from subject

In [ ]:
null_from_index = train.loc[train['clean_subject'].isnull() == True].index


In [ ]:
null_from_index

Int64Index([   20,    26,    27,    46,    47,    51,    80,    93,   120,
              121,
            ...
            10709, 10771, 10781, 10793, 10809, 10817, 10829, 10834, 10859,
            10879],
           dtype='int64', length=587)

In [ ]:
for i in null_from_index:
  train.loc[i,'clean_subject']="NOSUB"

In [ ]:
train.iloc[26]['clean_subject']

'NOSUB'

removing null from body

In [ ]:
null_from_index = train.loc[train['clean_esc_body'].isnull() == True].index


In [ ]:
null_from_index

Int64Index([10895, 10980], dtype='int64')

In [ ]:
for i in null_from_index:
  train.loc[i,'clean_esc_body']="NOBOD"

In [ ]:
train.iloc[10895]['clean_esc_body']

'NOBOD'

#Validating the model on the training dataa


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train['clean_subject']
y = train['Label']
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X = train['clean_esc_body']
y = train['Label']
X_train_bod, X_test_bod, y_train_bod, y_test_bod = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range = (1, 2),max_features=6000)
tfidf_sub = tfidf_vectorizer.fit_transform(X_train_sub)
tfidf_bod = tfidf_vectorizer.fit_transform(X_train_bod)

In [ ]:
tfidf_test_sub = tfidf_vectorizer.transform(X_test_sub)
tfidf_test_bod = tfidf_vectorizer.transform(X_test_bod)

In [ ]:
dup_train_features = np.hstack([
    tfidf_sub.toarray(),
    tfidf_bod.toarray()])
# to concatenate the features

In [ ]:
dup_test_features = np.hstack([
    tfidf_test_sub.toarray(),
    tfidf_test_bod.toarray()])

In [ ]:
dup_train_features.shape

(8785, 12000)

In [ ]:
dup_test_features.shape

(2197, 12000)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

In [ ]:
lr = LogisticRegression()
weights = np.linspace(0, 0.99, 100)# initialize random weights
param_grid = {'class_weight' : [{0:x, 1: 1.0-x} for x in weights], 'C' : [0.01, 0.1, 1, 10, 100]}
gridsearch = GridSearchCV(estimator= lr, 
                          param_grid= param_grid,
                          cv=3, 
                          n_jobs=-1, 
                          scoring='f1', 
                          verbose=2).fit(dup_train_features, y_train_bod)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
gridsearch.best_params_

{'C': 100, 'class_weight': {0: 0.12, 1: 0.88}}

In [ ]:
logreg = LogisticRegression(solver = 'newton-cg', C = 100, class_weight = {0: 0.12, 1: 0.88})
logreg.fit(dup_train_features,y_train_bod)
prediction = logreg.predict(dup_test_features)
f1_score(y_test_bod, prediction)

0.9744525547445254

### This Model gave a Kaggle score on 0.56 and further models based on the tfidf features gave accuracy around the 0.67-0.82